# Summerize Shareholding:

#### Summerize: 

- For all stocks from RawReports folder, read specific Excels sheet into dataframe  
- Add stock name column to df 
- Transform the df 
- Consolidate the df 

#### Analyze:

- Compute something
- Write the computed new data 
- Analyze or visualize the new data 

#### Summary Report File Name Convention: 
- Summary01-Top-Ratios 
- Summary02-Quarterly-Results
- Summary03-Profit-And-Loss
- Summary04-Compounded-Sales-Growth
- Summary05-Compounded-Profit-Growth
- Summary06-Stock-Price-CAGR 
- Summary07-Return-on-Equity
- Summary08-Balance-Sheet
- Summary09-Cash-Flows
- Summary10-Ratios
- Summary11-Shareholding-Pattern

## 1.1 Imports:

In [1]:
import requests as rq
import pandas as pd 
import time
import datetime
from bs4 import BeautifulSoup
import re
import os


## 1.2 Define Inputs:

>  ⚠️ <span style="color:red"> **Update the CSV File below to read as needed:**</span>

In [2]:
## CAREFULLY UPDATE THIS. This should be same as what specified in RawReport notebook

# For My Watchlisted stocks
input_cvs_file = "../Input/" + "stocks-watchlisted.csv"
# For PSU stocks
input_cvs_file = "../Input/" + "stocks-psu.csv"

input_cvs_file = "../Input/" + "stocks-psu.csv"

## UPDATE THIS. The folder is in  ../Output/RawReports/. This is where raw reports are read and summerized

raw_report_folder_name     = "20240816-085421"


In [3]:
raw_report_folder_path = "../Output/RawReports/" + raw_report_folder_name

df = pd.read_csv(input_cvs_file)
df.head(5)


,Screener Stock Symbol,Url Segment
0,BANKBARODA,Consolidated
1,BANKINDIA,Consolidated
2,BDL,Null
3,BEL,Consolidated
4,BHEL,Consolidated


## 1.3 Process reports:

In [4]:
df_shareholding_pattern = pd.DataFrame([])

for index, row in df.iterrows():
    stock_symbol = row["Screener Stock Symbol"]
    filepath = raw_report_folder_path + "/" + stock_symbol + ".xlsx"
    df_data =  pd.read_excel(filepath, sheet_name="Shareholding Pattern")
    print("Processing stock ",stock_symbol)
    # Add stockname column and move into 2nd position: 
    df_data['Stockname'] = stock_symbol
    column_to_move = df_data.pop("Stockname")
    df_data.insert(1, "Stockname", column_to_move)
    df_shareholding_pattern = pd.concat([df_shareholding_pattern, df_data], ignore_index=True, sort=False)
    
print("All done!")    

Processing stock  BANKBARODA
Processing stock  BANKINDIA
Processing stock  BDL
Processing stock  BEL
Processing stock  BHEL
Processing stock  BPCL
Processing stock  CANBK
Processing stock  CENTRALBK
Processing stock  COALINDIA
Processing stock  COCHINSHIP
Processing stock  CONCOR
Processing stock  ENGINERSIN
Processing stock  GAIL
Processing stock  GICRE
Processing stock  GUJGASLTD
Processing stock  HAL
Processing stock  HINDCOPPER
Processing stock  HINDPETRO
Processing stock  HUDCO
Processing stock  INDIANB
Processing stock  IOB
Processing stock  IOC
Processing stock  IRCON
Processing stock  IRCTC
Processing stock  IRFC
Processing stock  ITI
Processing stock  KIOCL
Processing stock  LICI
Processing stock  MAHABANK
Processing stock  MAZDOCK
Processing stock  MIDHANI
Processing stock  MMTC
Processing stock  MRPL
Processing stock  NATIONALUM
Processing stock  NBCC
Processing stock  NHPC
Processing stock  NIACL
Processing stock  NLCINDIA
Processing stock  NMDC
Processing stock  NTPC
Proce

## 1.5 Inspect dataframe:

In [5]:
print(len(df_shareholding_pattern))

302


In [6]:
df_shareholding_pattern.describe()

,Shareholding Pattern,Stockname,Sep 2021,Dec 2021,Mar 2022,Jun 2022,Sep 2022,Dec 2022,Mar 2023,Jun 2023,Sep 2023,Dec 2023,Mar 2024,Jun 2024,Jul 2024
count,302,302,291,297,297,302,302,302,302,302,302,302,302,302,6
unique,7,56,267,272,275,282,285,286,279,283,286,283,283,279,6
top,Promoters,BPCL,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,56.66%
freq,56,7,11,12,14,13,8,7,6,7,5,6,4,6,1


In [7]:
df_shareholding_pattern.columns

Index(['Shareholding Pattern', 'Stockname', 'Sep 2021', 'Dec 2021', 'Mar 2022',
       'Jun 2022', 'Sep 2022', 'Dec 2022', 'Mar 2023', 'Jun 2023', 'Sep 2023',
       'Dec 2023', 'Mar 2024', 'Jun 2024', 'Jul 2024'],
      dtype='object')

## 1.6 Sort column ordering:

In [8]:
date = '1970-01-01'
df_shareholding_pattern = df_shareholding_pattern.sort_index(
    axis=1, 
    key=lambda x: pd.to_datetime(x, errors='coerce', format='mixed').fillna(date)
)

In [9]:
df_shareholding_pattern.columns

Index(['Shareholding Pattern', 'Stockname', 'Sep 2021', 'Dec 2021', 'Mar 2022',
       'Jun 2022', 'Sep 2022', 'Dec 2022', 'Mar 2023', 'Jun 2023', 'Sep 2023',
       'Dec 2023', 'Mar 2024', 'Jun 2024', 'Jul 2024'],
      dtype='object')

In [10]:
df_shareholding_pattern.head(5)

,Shareholding Pattern,Stockname,Sep 2021,Dec 2021,Mar 2022,Jun 2022,Sep 2022,Dec 2022,Mar 2023,Jun 2023,Sep 2023,Dec 2023,Mar 2024,Jun 2024,Jul 2024
0,Promoters,BANKBARODA,63.97%,63.97%,63.97%,63.97%,63.97%,63.97%,63.97%,63.97%,63.97%,63.97%,63.97%,63.97%,NaN
1,FIIs,BANKBARODA,7.82%,7.37%,9.14%,8.23%,8.93%,9.97%,10.93%,12.29%,12.39%,12.27%,12.40%,11.45%,NaN
2,DIIs,BANKBARODA,14.70%,14.65%,15.08%,16.85%,18.64%,17.69%,16.74%,15.77%,15.74%,15.75%,16.04%,15.77%,NaN
3,Government,BANKBARODA,0.00%,0.00%,0.00%,0.00%,0.26%,0.26%,0.26%,0.26%,0.26%,0.26%,0.26%,0.26%,NaN
4,Public,BANKBARODA,13.50%,14.00%,11.81%,10.95%,8.21%,8.12%,8.10%,7.70%,7.64%,7.76%,7.32%,8.55%,NaN


## 1.7 Save summary report:

In [12]:
output_path = "../Output/SummaryReports/"+"Summary11-Shareholding-Pattern-"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+".csv"

df_shareholding_pattern.to_csv(output_path,index=False,sep=",")